# Data Preprocessing

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/labeled_119.csv', index_col = 'Unnamed: 0')
df['dialogue'] = df['dialogue'].str.replace('\n', ' ', regex=False)
df.head()

,filePath,urgencyLevel,disasterLarge,disasterMedium,symptoms,triage,dialogue,label
68238,/hdd/data/088.위급상황_음성-음향_고도화_119_지능형_신고접수_음성_인...,상,구급,질병(중증 외),['복통'],응급증상,"수보자: 1, 119입니다. 신고자: 아, 여기 저희 아버지가 암 환자신데요 수보자...",2
62810,/hdd/data/088.위급상황_음성-음향_고도화_119_지능형_신고접수_음성_인...,하,구급,부상,[],NaN,"신고자: 네, 아이가 놀다가 팔이 탈골이 된 거 같은데요. 수보자: 아 그래요? 계...",2
61283,/hdd/data/088.위급상황_음성-음향_고도화_119_지능형_신고접수_음성_인...,하,구급,질병(중증 외),['요통'],잠재응급증상,"수보자: 예, 119입니다. 신고자: 네, 여보세요. 수보자: 예, 말씀하세요. 신...",1
53628,/hdd/data/088.위급상황_음성-음향_고도화_119_지능형_신고접수_음성_인...,하,구급,질병(중증),"['복통', '기타통증', '호흡곤란', '그밖의통증기타', '어지러움']",잠재응급증상,"수보자: 네, 119 상황실입니다. 신고자: 여보세요. 수보자: 여보세요. 신고자:...",2
103449,/hdd/data/088.위급상황_음성-음향_고도화_119_지능형_신고접수_음성_인...,중,구조,대물사고,[],NaN,"신고자: 네, 119 수보자: 119입니다. 신고자: 예, 일로 양산동인데요. 수보...",2


In [ ]:
# 각 라벨에서 500개씩 샘플링
df_train = df.groupby('label').apply(lambda x: x.sample(n=500, random_state=526))

df_train = df_train[['dialogue', 'label']]
# 인덱스 초기화
df_train = df_train.reset_index(drop=True)
df_train.head()

/tmp/ipykernel_1497446/3609070347.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train = df.groupby('label').apply(lambda x: x.sample(n=500, random_state=526))


,dialogue,label
0,"수보자: 입니다. 신고자: 네, 안녕하세요. 수보자: 네, 말씀하세요. 신고자: 저...",0
1,"수보자: 119입니다. 신고자: 네, 여기 현대 동궁아파트 신고자: [개인정보] 수...",0
2,"수보자: 네 여보세요. 신고자: 예 예예. 수보자: 예, 가고 있는데 사람은 다 나...",0
3,"수보자: 네 119입니다. 신고자: 예, 안녕하세요. 여기 강남구 대치동, 신고자:...",0
4,"수보자: 네, 119입니다. 신고자: 여보세요? 수보자: 네 신고자: 아, 저희 집...",0


In [ ]:
# 각 라벨에서 500개씩 샘플링
df_test = df.groupby('label').apply(lambda x: x.sample(n=50, random_state=526))

df_test = df_test[['dialogue', 'label']]
# 인덱스 초기화
df_test = df_test.reset_index(drop=True)
df_test.head()

/tmp/ipykernel_1497446/3024197611.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_test = df.groupby('label').apply(lambda x: x.sample(n=50, random_state=526))


,dialogue,label
0,"수보자: 입니다. 신고자: 네, 안녕하세요. 수보자: 네, 말씀하세요. 신고자: 저...",0
1,"수보자: 119입니다. 신고자: 네, 여기 현대 동궁아파트 신고자: [개인정보] 수...",0
2,"수보자: 네 여보세요. 신고자: 예 예예. 수보자: 예, 가고 있는데 사람은 다 나...",0
3,"수보자: 네 119입니다. 신고자: 예, 안녕하세요. 여기 강남구 대치동, 신고자:...",0
4,"수보자: 네, 119입니다. 신고자: 여보세요? 수보자: 네 신고자: 아, 저희 집...",0


# Modeling

In [ ]:
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

/home/ubuntu/work_space/emerg_classifi/.emerg_classifi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_name = 'klue/bert-base'

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load KoBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)  # Assuming 3 labels

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 레이어 고정하기
for name, param in model.named_parameters():
    if "bert.embeddings" in name:
        param.requires_grad = False  # 임베딩 레이어 고정
    elif "bert.encoder.layer." in name:
        layer_num = int(name.split(".")[3])  # 레이어 번호 추출
        if layer_num <= 6:  # 0-5 레이어 고정
            param.requires_grad = False
        else:  # 6-11 레이어 학습 가능
            param.requires_grad = True
    else:
        param.requires_grad = True  # 풀링 레이어 및 분류기는 학습 가능

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['dialogue'], padding='max_length', truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 150/150 [00:00<00:00, 447.26 examples/s]


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 평가를 위한 지표 계산 함수 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # 예측된 클래스 가져오기
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from transformers import TrainingArguments
# from torch.optim import AdamW

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True
)

/home/ubuntu/work_space/emerg_classifi/.emerg_classifi/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer, EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,  # 지표 계산 함수 추가
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.058200,0.936544,0.613333,0.616063,0.654002,0.613333
2,0.884200,0.740516,0.653333,0.643345,0.641151,0.653333
3,0.779300,0.704988,0.680000,0.653155,0.674501,0.680000
4,0.728100,0.640812,0.693333,0.667611,0.693815,0.693333
5,0.681900,0.609205,0.733333,0.731632,0.731158,0.733333
6,0.642600,0.540569,0.773333,0.771405,0.771395,0.773333
7,0.626100,0.521414,0.800000,0.800559,0.807019,0.800000
8,0.597600,0.499275,0.806667,0.801024,0.804069,0.806667
9,0.568400,0.486429,0.813333,0.809810,0.810321,0.813333
10,0.570500,0.475823,0.806667,0.803351,0.803810,0.806667


TrainOutput(global_step=240, training_loss=0.7136807362238566, metrics={'train_runtime': 296.5056, 'train_samples_per_second': 50.589, 'train_steps_per_second': 0.809, 'total_flos': 3946701265920000.0, 'train_loss': 0.7136807362238566, 'epoch': 10.0})

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 평가를 위한 지표 계산 함수 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # 예측된 클래스 가져오기
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 평가
eval_results = trainer.evaluate()

# 평가 결과 출력
print("Evaluation Results:", eval_results)

Evaluation Results: {'eval_loss': 0.47582340240478516, 'eval_accuracy': 0.8066666666666666, 'eval_f1': 0.8033507292921184, 'eval_precision': 0.8038095238095238, 'eval_recall': 0.8066666666666666, 'eval_runtime': 1.5055, 'eval_samples_per_second': 99.632, 'eval_steps_per_second': 1.993, 'epoch': 10.0}


In [ ]:
output_dir = './saved_model'
# 모델 저장
model.save_pretrained(output_dir)
# 토크나이저 저장
tokenizer.save_pretrained(output_dir)

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')